In [1]:
using TensorFlow
tf_version()

v"1.10.0"

# Variables

A TensorFlow **variable** is the best way to represent shared, persistent state
manipulated by your program.

Variables are manipulated via the `Variable` class. A `Variable`
represents a tensor whose value can be changed by running ops on it. Unlike
`Tensor` objects, a `Variable` exists outside the context of a single
`run` call.

Internally, a `Variable` stores a persistent tensor. Specific ops allow you
to read and modify the values of this tensor. These modifications are visible
across multiple `Session`s, so multiple workers can see the same values for a
`Variable`.

## Creating a Variable

The best way to create a variable is to call the `get_variable`
function. This function requires you to specify the `Variable`'s name. This name
will be used by other replicas to access the same variable, as well as to name
this variable's value when checkpointing and exporting models. `get_variable`
also allows you to reuse a previously created variable of the same name, making it
easy to define models which reuse layers.

To create a variable with `get_variable`, simply provide the name, shape and dtype:

In [2]:
my_variable = get_variable("my_variable", [1,2,3], Float32)

Variable{Float32}(<Tensor my_variable:1 shape=(1, 2, 3) dtype=Float32>, <Tensor my_variable/Assign:1 shape=(1, 2, 3) dtype=Float32>)

This creates a variable named "my_variable" which is a three-dimensional tensor
with shape `[1, 2, 3]` and dtype `Float32`. Its initial value will be randomized via
`TensorFlow.Variables.NormalInitializer`.

Usually the `Variable`'s name is the same as the name of the object we choose to reference the `Variable`, as in the previous example. TensorFlow.jl provides a syntatic sugar for this pattern using the `@tf` macro:

```julia
@tf my_variable = get_variable([1,2,3], Float32) # same as the definition above
```

The macro also works on any assignment expression where the right hand side is a `Tensor`. For example, `@tf i = constant(1)` is the same as writing `i = constant(1, name="i")`.

You may optionally specify the initializer to `get_variable`. For example:

In [3]:
@tf my_int_variable = get_variable([1,2,3], Int64; initializer=ConstantInitializer(0))

Variable{Int64}(<Tensor my_int_variable:1 shape=(1, 2, 3) dtype=Int64>, <Tensor my_int_variable/Assign:1 shape=(1, 2, 3) dtype=Int64>)

When initialized, all cells of `my_int_variable` will be zero. You may also pass any univariate distributions from Distributions.jl as the initializer.

**⋆Note**: Initialization using the value of another `Tensor` is not yet supported in TensorFlow.jl.

### Variable collections

Because disconnected parts of a TensorFlow program might want to create
variables, it is sometimes useful to have a single way to access all of
them. For this reason TensorFlow provides **collections**, which are named lists
of tensors or other objects, such as `Variable` instances.

By default every `Variable` gets placed in the following two collections in the default graph:

 * `Variables` --- all defined variable.
 * `TrainableVariables` --- variables for which TensorFlow will
   calculate gradients.

To retrieve a list of all the variables (or other objects) you've placed in a collection you can use `get_collection`:

In [4]:
vars = get_collection(:Variables)
trainable_vars = get_collection(:TrainableVariables);

If you don't want a variable to be trainable, specify `trainable=false` as a keyword argument to `get_variable`:

In [5]:
@tf my_non_trainable = get_variable([], Float32; trainable=false)
println(my_non_trainable in vars)
println(my_non_trainable in trainable_vars)

true
false


**⋆Note**: Unlike the Python API, TensorFlow.jl does not keep separate `GLOBAL_VARIABLES` and `LOCAL_VARIABLES` collections.

You can also use your own collections. Any `Symbol` is a valid collection name,
and there is no need to explicitly create a collection. To add a variable (or
any other object) to a collection after creating the variable, call
`TensorFlow.add_to_collection`.  For example, the following code adds an existing
variable named `my_local` to a collection named `my_collection_name`:

In [6]:
@tf my_local = get_variable([], Float64)
TensorFlow.add_to_collection(:my_collection, my_local)
my_local in get_collection(:my_collection)

true

### Device placement [WIP]

## Initializing variables

Before you can use a variable, it must be initialized. If you are programming in
the low-level TensorFlow API (that is, you are explicitly creating your own
graphs and sessions), you must explicitly initialize the variables.  Most
high-level frameworks such as `contrib.slim`, `estimator.Estimator` and
`Keras` automatically initialize variables for you before training a model.

Explicit initialization is otherwise useful because it allows you not to rerun
potentially expensive initializers when reloading a model from a checkpoint as
well as allowing determinism when randomly-initialized variables are shared in a
distributed setting.

To initialize all trainable variables in one go, before training starts, call
`global_variables_initializer()`. This function returns a single operation
responsible for initializing all variables in the
`Variables` collection. Running this operation initializes
all variables. For example:

In [7]:
sess = Session()
run(sess, global_variables_initializer())
# Now all variables are initialized

2018-10-04 13:50:20.654869: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


If you do need to initialize variables yourself, you can run the variable's initializer operation via its assign node. For example:

In [8]:
run(sess, my_variable.assign_node);

**⋆Note**: `report_unitialized_variables` and `initialized_value` not yet supported in TensorFlow.jl.

## Using Variables

To use the value of a `Variable` in a TensorFlow graph, simply treat it like
a normal `Tensor`:

In [9]:
@tf v = get_variable([], Float64; initializer=ConstantInitializer(0.0))
w = v + 1 # w is a Tensor which is computed based on the value of v.
          # Any time a variable is used in an expression it gets automatically
          # converted to a Tensor representing its value.

<Tensor Add:1 shape=() dtype=Float64>

To assign a value to a variable, use the methods `assign`, `assign_add`, and
friends in the `Variable` class. For example, here is how you can call these
methods:

In [10]:
assignment = assign_add(v, 1)
run(sess, global_variables_initializer())
run(sess, assignment)

1.0

Most TensorFlow optimizers have specialized ops that efficiently update the
values of variables according to some gradient descent-like algorithm. See
`train.Optimizer` for an explanation of how to use optimizers.

**⋆Note**: `read_value` not yet supported by TensorFlow.jl

## Sharing variables

TensorFlow supports two ways of sharing variables:

 * Explicitly passing `Variable` objects around.
 * Implicitly wrapping `Variable` objects within `variable_scope` objects.

While code which explicitly passes variables around is very clear, it is
sometimes convenient to write TensorFlow functions that implicitly use
variables in their implementations. Most of the functional layers from
`layers` use this approach, as well as all `metrics`, and a few other
library utilities.

Variable scopes allow you to control variable reuse when calling functions which
implicitly create and use variables. They also allow you to name your variables
in a hierarchical and understandable way.

For example, let's say we write a function to create a convolutional / relu
layer:

In [11]:
using Distributions: Normal

function conv_relu(input, kernel_shape, bias_shape)
    # Create variable named "weights"
    @tf weights = get_variable(kernel_shape, Float64; initializer=Normal())
    # Create variable named "biases"
    @tf biases = get_variable(bias_shape, Float64; initializer=ConstantInitializer(0.0))
    conv = nn.conv2d(input, weights, [1,1,1,1], "SAME")
    return nn.relu(conv + biases)
end

conv_relu (generic function with 1 method)

This function uses short names `weights` and `biases`, which is good for
clarity. In a real model, however, we want many such convolutional layers, and
calling this function repeatedly would not work:

In [12]:
input1 = random_normal([1,10,10,32])
input2 = random_normal([1,20,20,32])
x = conv_relu(input1, [5,5,32,32], [32])
# x = conv_relu(x, [5,5,32,32], [32]) # This fails.

<Tensor Relu:1 shape=(1, 10, 10, 32) dtype=Float64>

Since the desired behavior is unclear (create new variables or reuse the
existing ones?) TensorFlow will fail. Calling `conv_relu` in different scopes,
however, clarifies that we want to create new variables:

In [13]:
function my_image_filter(input_images)
    variable_scope("conv1") do
        # Variables created here will be named "conv1/weights", "conv1/biases".
        global relu1 = conv_relu(input_images, [5,5,32,32], [32])
    end
    variable_scope("conv2") do
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5,5,32,32], [32])
    end
end

my_image_filter (generic function with 1 method)

If you do want the variables to be shared, you can create a scope with the same name using `reuse=true`:

In [14]:
variable_scope("model") do
    output1 = my_image_filter(input1)
end
variable_scope("model", reuse=true) do
    output2 = my_image_filter(input2)
end

<Tensor Relu_5:1 shape=(1, 20, 20, 32) dtype=Float64>

**⋆Note**: `scope.reuse_variables` and passing exising scopes to `variable_scope` not yet supported by TensorFlow.jl

----------------------
*Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 3.0 License](https://creativecommons.org/licenses/by/3.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). This notebook is adapted from the official [TensorFlow Guide on Variables](https://www.tensorflow.org/guide/variables).*